# Seminario sobre Python - Pandas

## Los datos: Encuesta Financiera de las Familias 2014 (BdE)

El seminario se va a basar en los datos de la __[EFF](https://www.bde.es/bde/es/areas/estadis/Otras_estadistic/Encuesta_Financi/)__. Se trata precisamente de un panel de unas 6.000 familias que se viene realizando desde el año 2.002 con el fin de obtener información directa sobre las condiciones financieras de las familias españolas. Permite la elaboración de estudios sobre las decisiones de inversión y financiación de las familias españolas y sobre su situación patrimonial, que resultan relevantes para el mejor conocimiento de nuestra economía y el adecuado diseño de las políticas públicas.

<img src="https://www.caixabank.es/deployedfiles/particulares/Estaticos/Imagenes/Segmentos/FotobannerFamilydestacadook-2.jpg" alt="pandas" title="family" width="400" />

Para este ejercicio vamos a centrarnos en la encuesta de 2014.

***

## Objetivos

- Familiarizarse con el proceso de datos, datos descriptivos en tablas y gráficos con Pandas
- Aplicar técnicas de Machine Learning (ML)


## Ejercicio

Vamos a analizar una campaña de márquetin de préstamos para la compra de automóviles. Primero se pide clasificar las familias según su propensión a compra de automóviles del último año. En segundo lugar, proponer una financiación personalizada que cubra la totalidad del valor del vehículo. 

Para saber qué familias han comprado un automóvil, tenemos la variable "p2_74" que representa el gasto total realizado para su adquisición. Cuando esta variable es positiva, significa ha adquirido un vehículo. Esta variable se encuentra en la tabla "otras_secciones_2014_imp1" (tabla A a partir de ahora). La tabla databol5 la llamaremos tabla B.

Para poder estimar la propensión se cuenta con el siguiente diccionario de variables:
- renta familiar (B.renthog13_eur14)
- edad del cabeza de familia (CF) (A.p1_2d_1)
- nivel de estudios del CF (B.neducdom): toma valores 1 a 3; inferior a bachillerato, bachillerato, estudios universitarios
- situación laboral del CF (B.nsitlabdom): toma valores 1 a 4; empleado por cuenta ajena, empleado por cuenta propia, jubilado, otro tipo de inactivo o parado 
- número de miembros del hogar (B.np1): toma valores 1 a 5; uno, dos, tres, cuatro, cinco o más
- niembros trabajando (B.nnumadtrab): toma valores 0 a 3; ninguno, uno, dos, tres o más

En resumen, lo que se pide es:
1. Estimar la propensión a compra de un automóvil en el último año
  - Opción 1:__[statsmodels glm](http://www.statsmodels.org/dev/examples/notebooks/generated/glm_formula.html)__
  - Opción 2:__[sklearn.ensemble.GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)__

2. Financiación a ofrecer: Estimar el percentil 90% de la distribución de precios de compra.
  - Opcion 1: __[statsmodels quantile_regression](http://www.statsmodels.org/dev/examples/notebooks/generated/quantile_regression.html)__
  - Opcion 1:__[sklearn.ensemble.GradientBoostingRegressor con loss='huber' o loss='quantile'.](http://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-quantile-py)__



La carpeta de trabajo es: 
> "L:\16630\Big Data Lab\3. Formacio\L. Formacion Interna\python\seminario1"

El cuestionario se puede en: 
> "eff/cuestionario_2014.pdf"

Las variables calculadar en databol estan en: 
> "eff/definitions_2014.doc"


### Carga de librerías

In [0]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
from statsmodels.compat import lmap
from scipy import stats
import patsy
import xgboost as xgb
import tensorflow as tf


import matplotlib.pyplot as plt   # librerias graficas 
import math  # funciones matemáticas
import re    # expresiones regulares 
import os    # funciones de sistema operativo
#os.chdir("L:/16630/Big Data Lab/3. Formacio/L. Formacion Interna/python/seminario1")
#os.chdir("C:/DL/pandas")

# habilitamos graficos para notebooks de Jupyter
%matplotlib inline

# Establecemos el límite de mostrar columnas a 200
pd.set_option("display.max_columns",200)

### Leemos los datos del paso previo

In [0]:
datos1 = pd.read_pickle("datos1_seminario.pkl")
datos1.shape

## Financiación a ofrecer: Estimar el percentil 90% de la distribución de precios de compra 

In [0]:
datos3 = datos1.loc[(datos1["auto"]==1) & (datos1["p2_74"]>0)].drop("auto", axis=1).copy()
datos3.shape


In [0]:
datos3 = datos3.dropna().copy()
datos3.shape

In [0]:
datos3.describe()

In [0]:
datos3.p2_74.hist(bins=20)  # edad
plt.title("gasto automóvil")
plt.xlabel("€")

In [0]:
datos3["LGasto"] = np.log(datos3.p2_74)
datos3.LGasto.hist(bins=20)  # gasto
plt.title("gasto automóvil")
plt.xlabel("€")

### Analizamos correlaciones

In [0]:
datos3[["p2_74","LGasto","renthog13_eur14","Lrenthog13_eur14","p1_2d_1","nsitlabdom","neducdom"]].corr()

In [0]:
sns.heatmap(datos3[["p2_74","renthog13_eur14","p1_2d_1","nsitlabdom","neducdom"]].corr(), vmin=-1, vmax=1 , cmap="YlGnBu")

### Descriptivo bivariante

In [0]:
lmp = sns.lmplot(  "Lrenthog13_eur14","p2_74",  datos3)
#ax = lmp.axes
#ax[0,0].set_xlim(0,500000)

In [0]:
sns.lmplot("p1_2d_1","p2_74",  datos3);

In [0]:
sns.lmplot("p1_2d_1","p2_74",  datos3 , order=2);   # polinomio orden 2

In [0]:
sns.lmplot( "neducdom","p2_74", datos3);

In [0]:
sns.stripplot(x='neducdom', y='p2_74', data=datos3, jitter=True)
sns.despine()

In [0]:
sns.stripplot(x='nsitlabdom', y='p2_74', data=datos3, jitter=True)
sns.despine()

## seleccionamos muestras

In [0]:
X_train, X_test = train_test_split(datos3, test_size=0.2, random_state=666)
print("N train: ", len(X_train),"N test: ",  len(X_test))

## modelo cuantiles

In [0]:
## regression robusta percentil 50%
mod1 = smf.quantreg('p2_74 ~ Lrenthog13_eur14', X_train)
res_1_50 = mod1.fit(q=.50)
print(res_1_50.summary())

Comparemos con la regression de minimos cuadrados: OLS

In [0]:
ols = smf.ols('p2_74 ~ Lrenthog13_eur14', X_train).fit()
ols.summary()

Veamos a hora distintos quantiles comparados

In [0]:
# p2_74 , LGasto
mod = smf.quantreg('p2_74 ~ Lrenthog13_eur14', X_train)
quantiles = np.arange(.05, .96, .1)
def fit_model(q):
    res = mod.fit(q=q)
    return [q, res.params['Intercept'], res.params['Lrenthog13_eur14']] + \
            res.conf_int().loc['Lrenthog13_eur14'].tolist()

models50 = pd.DataFrame([fit_model(x) for x in [0.5]], columns=['q', 'a', 'b','lb','ub'])
models90 = pd.DataFrame([fit_model(x) for x in [0.9]], columns=['q', 'a', 'b','lb','ub'])

models = [fit_model(x) for x in quantiles]
models = pd.DataFrame(models, columns=['q', 'a', 'b','lb','ub'])


ols = smf.ols('p2_74 ~ Lrenthog13_eur14', X_train).fit()
ols_ci = ols.conf_int().loc['Lrenthog13_eur14'].tolist()
ols = dict(a = ols.params['Intercept'],
           b = ols.params['Lrenthog13_eur14'],
           lb = ols_ci[0],
           ub = ols_ci[1])

print(models50)
print(models)
print(ols)

Ahora graficamos los distintos cuantiles y la regression OLS:

In [0]:
models.shape[0]

In [0]:
x = np.arange(X_train.Lrenthog13_eur14.min(), X_train.Lrenthog13_eur14.max()
              , (X_train.Lrenthog13_eur14.max() - X_train.Lrenthog13_eur14.min())/50.)
xexp = np.exp(x)
get_y = lambda a, b: a + b * x

fig, ax = plt.subplots(figsize=(8, 6))

for i in range(models.shape[0]):
    y = get_y(models.a[i], models.b[i])
    ax.plot(xexp, y, linestyle='dotted', color='grey')

y = get_y(models90['a'][0], models90['b'][0])
ax.plot(xexp, y, color='green', label='q90')

y = get_y(models50['a'][0], models50['b'][0])
ax.plot(xexp, y, color='blue', label='q50')

y = get_y(ols['a'], ols['b'])
ax.plot(xexp, y, color='red', label='OLS')

ax.scatter(X_train.renthog13_eur14, X_train.p2_74, alpha=.2)
#ax.set_xlim((240, 3000))
#ax.set_ylim((240, 2000))
legend = ax.legend()
plt.title('Financiación a ofrecer')
ax.set_xlabel('Log Renta', fontsize=16)
ax.set_ylabel('Gasto Automóvil', fontsize=16);

Adaptamos el eje x a la realidad de la población  8O

In [0]:
x = np.arange(X_train.Lrenthog13_eur14.min(), X_train.Lrenthog13_eur14.max()
              , (X_train.Lrenthog13_eur14.max() - X_train.Lrenthog13_eur14.min())/50.)
xexp = np.exp(x)
get_y = lambda a, b: a + b * x

fig, ax = plt.subplots(figsize=(8, 6))

for i in range(models.shape[0]):
    y = get_y(models.a[i], models.b[i])
    ax.plot(xexp, y, linestyle='dotted', color='grey')

y = get_y(models90['a'][0], models90['b'][0])
ax.plot(xexp, y, color='green', label='q90')

y = get_y(models50['a'][0], models50['b'][0])
ax.plot(xexp, y, color='blue', label='q50')

y = get_y(ols['a'], ols['b'])
ax.plot(xexp, y, color='red', label='OLS')

ax.scatter(X_train.renthog13_eur14, X_train.p2_74, alpha=.2)
ax.set_xlim((0, 100000))
ax.set_ylim((0,  65000))

legend = ax.legend()
plt.title('Financiación a ofrecer')
ax.set_xlabel('Log Renta', fontsize=16)
ax.set_ylabel('Gasto Automóvil', fontsize=16);

### Múltiples variables

In [0]:
## regression robusta percentil 50%
mod = smf.quantreg('p2_74 ~ Lrenthog13_eur14 + p1_2d_1 + C(neducdom)', X_train)
res50 = mod.fit(q=.50)
print(res50.summary())

In [0]:
## regression robusta percentil 99%
res90 = mod.fit(q=.90)
print(res90.summary())

In [0]:
X_train_res = X_train.copy()
X_train_res["pred50"] = res50.predict(X_train)
X_train_res["pred90"] = res90.predict(X_train)

X_test_res = X_test.copy()
X_test_res["pred50"] = res50.predict(X_test)
X_test_res["pred90"] = res90.predict(X_test)

In [0]:
X_test_res[["p2_74","pred50","pred90"]].head()

In [0]:
X_test_res[["p2_74","pred50","pred90"]].describe(percentiles=[.25,.5,.75,.9])

Verificamos como queda el percentil 90% de la muestra de test

In [0]:
print("percentil en train: ", np.round(100* np.mean(np.where(X_train_res["p2_74"]<X_train_res["pred90"],1,0)),1))
print("percentil en test: ", np.round(100* np.mean(np.where(X_test_res["p2_74"]<X_test_res["pred90"],1,0)),1))

## Veamos ahora la estimación del percentil con el modelo GBM

In [0]:
varsModel = ["Lrenthog13_eur14"]
X1_train, X1_test = X_train[varsModel], X_test[varsModel]
Y1_train, Y1_test = X_train["p2_74"], X_test["p2_74"]

In [0]:
x = np.arange(X_train.Lrenthog13_eur14.min(), X_train.Lrenthog13_eur14.max()
              , (X_train.Lrenthog13_eur14.max() - X_train.Lrenthog13_eur14.min())/50.)
xx = pd.DataFrame(x, columns=["Lrenthog13_eur14"])

### seleccionamos el cuantil
alpha = 0.90

clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=20, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)

clf.fit(X1_train, Y1_train)

# Make the prediction on the meshed x-axis
y_upper = clf.predict(xx)

clf.set_params(alpha=1.0 - alpha)
clf.fit(X1_train, Y1_train)

# Make the prediction on the meshed x-axis
y_lower = clf.predict(xx)

clf.set_params(alpha=0.5)
clf.fit(X1_train, Y1_train)

# Make the prediction on the meshed x-axis
y_pred50 = clf.predict(xx)

clf.set_params(loss='ls')
clf.fit(X1_train, Y1_train)

# Make the prediction on the meshed x-axis
y_pred = clf.predict(xx)


# Plot the function, the prediction and the 90% confidence interval based on
# the MSE
fig = plt.figure()
#plt.plot(X1_test, Y1_test, 'g:', label=u'$f(x) = x\,\sin(x)$')
plt.plot(np.exp(X1_test), Y1_test, 'b.', markersize=10, label=u'Observations')
plt.plot(np.exp(xx), y_pred, 'r-', label=u'Prediction')
plt.plot(np.exp(xx), y_pred50, 'b-', label=u'Prediction 50%')
plt.plot(np.exp(xx), y_upper, 'k-')
plt.plot(np.exp(xx), y_lower, 'k-')
plt.fill(np.concatenate([np.exp(xx), np.exp(xx)[::-1]]),
         np.concatenate([y_upper, y_lower[::-1]]),
         alpha=.5, fc='b', ec='None', label='90% prediction interval')
plt.xlabel('Log Renta')
plt.ylabel('Gasto')
plt.xlim(0, 100000)
plt.ylim(0, 60000)
plt.legend(loc='upper left')
plt.show()

Ahora con un modelo completo

In [0]:
varsModel = ["Lrenthog13_eur14","p1_2d_1","neducdom"]
X1_train, X1_test = X_train[varsModel], X_test[varsModel]
Y1_train, Y1_test = X_train["p2_74"], X_test["p2_74"]

In [0]:
X1_test.shape

In [0]:
alpha = 0.90

clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=20, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)

clf.fit(X1_train, Y1_train)



In [0]:
X_train_res = X_train.copy()

X_train_res["pred90"] = clf.predict(X1_train)

X_test_res = X_test.copy()

X_test_res["pred90"] = clf.predict(X1_test)


In [0]:
alpha = 0.50

clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=20, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)

clf.fit(X1_train, Y1_train)


In [0]:
X_train_res["pred50"] = clf.predict(X1_train)
X_test_res["pred50"] = clf.predict(X1_test)


In [0]:
X_test_res[["p2_74","pred50","pred90"]].head()

In [0]:

X_test_res[["p2_74","pred50","pred90"]].describe(percentiles=[.25,.5,.75,.9])

Verificamos como queda el percentil 90% de la muestra de test

In [0]:
print("percentil en train: ", np.round(100* np.mean(np.where(X_train_res["p2_74"]<X_train_res["pred90"],1,0)),1))
print("percentil en test: ", np.round(100* np.mean(np.where(X_test_res["p2_74"]<X_test_res["pred90"],1,0)),1))

#### autor: Ferran Carrascosa 8-6-2018.